 DOA CNN      
====================

 Net     : DenseNet     
 Train_Dataset : Normalized Original 800 + SpecAugment(Freq, Time Masking & Warping) 800 X 3 ===> Total : 3200    
 Masking == Mean of two magnitude    
 Test_Dataset : Normalized Original 200 & Contest testing data 500
 
 *(Without Phase)*    
 Epochs = 100    
 Batch size = 64   
 Optimizer = Adam   
 
 

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor
from collections import OrderedDict
from torch.utils.data import DataLoader, Dataset

In [2]:
training_epochs = 100
learning_rate = 1e-3
batch_size = 64

In [3]:
filename = 'specaug_dataset.npz'
dataset = np.load(filename)

In [4]:
inputs_train = dataset['inputs_train']
inputs_test  = dataset['inputs_test']
labels_train = dataset['labels_train']
labels_test  = dataset['labels_test']

In [5]:
print(inputs_train.shape)
print(labels_train.shape)
print(inputs_test.shape)
print(labels_test.shape)

(3200, 2, 257, 382)
(3200,)
(200, 2, 257, 382)
(200,)


In [6]:
class Custom_dataset(Dataset):
    
    def __init__(self, inputs, labels):
        self.inputs = torch.from_numpy(inputs).float()
        self.labels = torch.from_numpy(labels).long()
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        inputs = self.inputs[index]
        labels = self.labels[index]
        return inputs, labels

In [7]:
print(inputs_train.shape)
print(inputs_test.shape)


(3200, 2, 257, 382)
(200, 2, 257, 382)


In [8]:
train_set = Custom_dataset(inputs = inputs_train,
                           labels = labels_train)
test_set  = Custom_dataset(inputs = inputs_test,
                           labels = labels_test)


In [9]:
print(train_set.inputs.shape)
print(train_set.labels.shape)
print(test_set.inputs.shape)
print(test_set.labels.shape)

torch.Size([3200, 2, 257, 382])
torch.Size([3200])
torch.Size([200, 2, 257, 382])
torch.Size([200])


In [10]:
train_loader = DataLoader(dataset=train_set,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True)
test_loader  = DataLoader(dataset=test_set,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=False)


In [11]:
class _DenseLayer(nn.Module):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate, memory_efficient=False):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
                                           growth_rate, kernel_size=1, stride=1,
                                           bias=False)),
        self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                                           kernel_size=3, stride=1, padding=1,
                                           bias=False)),
        self.drop_rate = float(drop_rate)
        self.memory_efficient = memory_efficient

    def bn_function(self, inputs):
        # type: (List[Tensor]) -> Tensor
        concated_features = torch.cat(inputs, 1)
        bottleneck_output = self.conv1(self.relu1(self.norm1(concated_features)))  # noqa: T484
        return bottleneck_output

    # todo: rewrite when torchscript supports any
    def any_requires_grad(self, input):
        # type: (List[Tensor]) -> bool
        for tensor in input:
            if tensor.requires_grad:
                return True
        return False


    # torchscript does not yet support *args, so we overload method
    # allowing it to take either a List[Tensor] or single Tensor
    def forward(self, input):  # noqa: F811
        if isinstance(input, Tensor):
            prev_features = [input]
        else:
            prev_features = input

        if self.memory_efficient and self.any_requires_grad(prev_features):
            if torch.jit.is_scripting():
                raise Exception("Memory Efficient not supported in JIT")

            bottleneck_output = self.call_checkpoint_bottleneck(prev_features)
        else:
            bottleneck_output = self.bn_function(prev_features)

        new_features = self.conv2(self.relu2(self.norm2(bottleneck_output)))
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate,
                                     training=self.training)
        return new_features

In [12]:
class _DenseBlock(nn.ModuleDict):
    _version = 2

    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate, memory_efficient=False):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient,
            )
            self.add_module('denselayer%d' % (i + 1), layer)

    def forward(self, init_features):
        features = [init_features]
        for name, layer in self.items():
            new_features = layer(features)
            features.append(new_features)
        return torch.cat(features, 1)

In [13]:
class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))

In [14]:
class DenseNet(nn.Module):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
        memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
          but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_
    """

    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0.3, num_classes=11, memory_efficient=False):

        super(DenseNet, self).__init__()

        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(2, num_init_features, kernel_size=7, stride=2,
                                padding=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
        ]))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient
            )
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features,
                                    num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm2d(num_features))

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)

        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out

In [15]:
GPU_NUM = 7
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print('Currernt cuda device ', torch.cuda.current_device())

if device.type == 'cuda':
    print(torch.cuda.get_device_name(GPU_NUM))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(GPU_NUM)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(GPU_NUM)/1024**3,1), 'GB')

Currernt cuda device  7
GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [16]:
# def densenet121(pretrained=False, progress=True, **kwargs):
#     r"""Densenet-121 model from
#     `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#         progress (bool): If True, displays a progress bar of the download to stderr
#         memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
#           but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_
#     """
#     return _densenet('densenet121', 32, (6, 12, 24, 16), 64, pretrained, progress,
#                      **kwargs)


# def densenet161(pretrained=False, progress=True, **kwargs):
#     r"""Densenet-161 model from
#     `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#         progress (bool): If True, displays a progress bar of the download to stderr
#         memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
#           but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_
#     """
#     return _densenet('densenet161', 48, (6, 12, 36, 24), 96, pretrained, progress,
#                      **kwargs)


# def densenet169(pretrained=False, progress=True, **kwargs):
#     r"""Densenet-169 model from
#     `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#         progress (bool): If True, displays a progress bar of the download to stderr
#         memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
#           but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_
#     """
#     return _densenet('densenet169', 32, (6, 12, 32, 32), 64, pretrained, progress,
#                      **kwargs)


# def densenet201(pretrained=False, progress=True, **kwargs):
#     r"""Densenet-201 model from
#     `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
#     Args:
#         pretrained (bool): If True, returns a model pre-trained on ImageNet
#         progress (bool): If True, displays a progress bar of the download to stderr
#         memory_efficient (bool) - If True, uses checkpointing. Much more memory efficient,
#           but slower. Default: *False*. See `"paper" <https://arxiv.org/pdf/1707.06990.pdf>`_
#     """
#     return _densenet('densenet201', 32, (6, 12, 48, 32), 64, pretrained, progress,
#                      **kwargs)

In [17]:
def _densenet(arch, growth_rate, block_config, num_init_features, pretrained, progress,
              **kwargs):
    net = DenseNet(growth_rate, block_config, num_init_features, **kwargs)
    return net


def densenet_custom(pretrained=False, progress=True, **kwargs):
    return _densenet('densenet_custom', 32, (4, 4, 4), 64, pretrained, progress, **kwargs)

In [18]:
net = densenet_custom().to(device)

In [19]:
net

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(2, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [21]:
def acc_check(Net, dataset, epoch):
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataset:
            
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = Net(inputs)
            
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc = (100 * correct / total)
        return acc

In [22]:
total_batch = len(train_loader)


for epoch in range(training_epochs):
    avg_cost = 0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        hypothesis = net(inputs)
        cost = criterion(hypothesis, labels)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    
    test_acc  = acc_check(net, test_loader, epoch)
    train_acc = acc_check(net, train_loader, epoch)
    print('[Epoch : {:>4}]'.format(epoch + 1, avg_cost))
    print('cost = {:>.7} \t Acc_Test  : {:>.3} \t Acc_Train : {:>.3}'.format(avg_cost, test_acc, train_acc))
print('Learning finished')

[Epoch :    1]
cost = 2.245696 	 Acc_Test  : 27.5 	 Acc_Train : 28.5
[Epoch :    2]
cost = 1.947722 	 Acc_Test  : 34.0 	 Acc_Train : 34.2
[Epoch :    3]
cost = 1.734241 	 Acc_Test  : 42.5 	 Acc_Train : 41.7
[Epoch :    4]
cost = 1.613559 	 Acc_Test  : 40.5 	 Acc_Train : 45.3
[Epoch :    5]
cost = 1.489082 	 Acc_Test  : 50.0 	 Acc_Train : 48.9
[Epoch :    6]
cost = 1.397769 	 Acc_Test  : 50.5 	 Acc_Train : 52.4
[Epoch :    7]
cost = 1.335099 	 Acc_Test  : 46.5 	 Acc_Train : 51.8
[Epoch :    8]
cost = 1.269928 	 Acc_Test  : 58.0 	 Acc_Train : 58.2
[Epoch :    9]
cost = 1.19574 	 Acc_Test  : 46.5 	 Acc_Train : 53.8
[Epoch :   10]
cost = 1.184048 	 Acc_Test  : 67.0 	 Acc_Train : 57.6
[Epoch :   11]
cost = 1.113378 	 Acc_Test  : 62.0 	 Acc_Train : 60.6
[Epoch :   12]
cost = 1.114486 	 Acc_Test  : 59.5 	 Acc_Train : 61.0
[Epoch :   13]
cost = 1.05567 	 Acc_Test  : 53.5 	 Acc_Train : 60.8
[Epoch :   14]
cost = 1.00043 	 Acc_Test  : 54.0 	 Acc_Train : 56.5
[Epoch :   15]
cost = 0.9843892 	 Acc

In [23]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 77 %


In [24]:
correct = 0
total = 0

with torch.no_grad():
    for data in train_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the train images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the train images: 97 %


### Contest Test Data

In [25]:
cont_filename = 'contest_dataset.npz'

In [26]:
cont_dataset = np.load(cont_filename)

In [28]:
inputs_test  = cont_dataset['inputs']
labels_test  = cont_dataset['labels']

In [29]:
test_set  = Custom_dataset(inputs = inputs_test,
                           labels = labels_test)

test_loader  = DataLoader(dataset=test_set,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=False)

In [33]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 28 %
